In [1]:
import random
import numpy as np
from copy import deepcopy
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Legion\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie przynajmniej jednej sekwencji edycji (dodanie, usunięcie, zmiana znaku), która pozwala w minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.

In [2]:
def levenshtein_matrix_dist(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros((size_x, size_y))
    for x in range(size_x):
        matrix[x, 0] = x
        
    for y in range(size_y):
        matrix[0, y] = y
    
    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(matrix[x-1, y] + 1, matrix[x-1, y-1], matrix[x, y-1] + 1)
                
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1)
                
    result = int(matrix[size_x - 1, size_y - 1])
    
    return int(matrix[-1][-1]), matrix

In [3]:
levenshtein_matrix_dist("los", "kloc")

(2,
 array([[0., 1., 2., 3., 4.],
        [1., 1., 1., 2., 3.],
        [2., 2., 2., 1., 2.],
        [3., 3., 3., 2., 2.]]))

2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez wskazanie kolejnych wersji pierwszego łańcucha, w których dokonywana jest określona zmiana. "Wizualizacja" może działać w trybie tekstowym.

In [4]:
def visualisation(seq1, seq2):
    size_x = len(seq1)
    size_y = len(seq2)
    matrix = levenshtein_matrix_dist(seq1, seq2)[1]
    d = {"change": [],
         "idx": []}
    
    i = size_x
    j = size_y
    while i > 0 or j > 0:
        n = matrix[i][j]
        if matrix[i][j-1]+1 == n:
            d["change"].append("dodawanie")
            j -= 1
        elif matrix[i-1][j]+1 == n:
            d["change"].append("usuwanie")
            i -= 1
        elif matrix[i-1][j-1] == n:
            d["change"].append("-")
            i -= 1
            j -= 1
        else:
            d["change"].append("zamienianie")
            i -= 1
            j -= 1
            
        d["idx"].append(j)
    
    d["change"].reverse()
    d["idx"].reverse()
    
    i = 0
    j = 0
    print(seq1)
    for change, idx in zip(d["change"], d["idx"]):
        if change == "dodawanie":
            print(f"{seq1[:j]}*{seq2[j]}*{seq1[j:]}  (dodanie litery {seq2[j]})")
            seq1 = seq1[:j] + seq2[j] + seq1[j:]
        elif change == "usuwanie":
            print(f"{seq1[:i]}-{seq1[i+1:]} (usunięcie litery {seq1[i]})")
            seq1 = seq1[:i] + seq1[i+1:]
            continue
        elif change == "zamienianie":
            print(f"{seq1[:i]}*{seq2[j]}*{seq1[i+1:]} (zamiana liter {seq1[i]}->{seq2[j]})")
            seq1 = seq1[:j] + seq2[j] + seq1[j+1:]
        i += 1
        j += 1
    print(seq1)

3. Przedstaw wynik działania algorytmu z p. 2 dla następujących par łańcuchów:
los - kloc;
Łódź - Lodz;
kwintesencja - quintessence;
ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG.

In [5]:
visualisation("los", "kloc")

los
*k*los  (dodanie litery k)
klo*c* (zamiana liter s->c)
kloc


In [6]:
visualisation("Łódź", "Lodz")

Łódź
*L*ódź (zamiana liter Ł->L)
L*o*dź (zamiana liter ó->o)
Lod*z* (zamiana liter ź->z)
Lodz


In [7]:
visualisation("kwintesencja", "quintessence")

kwintesencja
*q*wintesencja (zamiana liter k->q)
q*u*intesencja (zamiana liter w->u)
quintes*s*encja  (dodanie litery s)
quintessenc*e*a (zamiana liter j->e)
quintessence- (usunięcie litery a)
quintessence


In [8]:
visualisation("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")

ATGAATCTTACCGCCTCG
ATGA*G*TCTTACCGCCTCG (zamiana liter A->G)
ATGAG*G*CTTACCGCCTCG (zamiana liter T->G)
ATGAGGCT*C*TACCGCCTCG  (dodanie litery C)
ATGAGGCTCT*G*CCGCCTCG (zamiana liter A->G)
ATGAGGCTCTG*G*CCGCCTCG  (dodanie litery G)
ATGAGGCTCTGGCC-CCTCG (usunięcie litery G)
ATGAGGCTCTGGCCCCT-G (usunięcie litery C)
ATGAGGCTCTGGCCCCTG


4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów

 - LCS

In [9]:
def longest_sub(seq1, seq2):
    m, n = len(seq1), len(seq2)
    dp = [[0] * (n+1) for _ in range(m+1)]

    for i in range(1, m+1):
        for j in range(1, n+1):
            if seq1[i-1] == seq2[j-1]:
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])

    lcs = []
    i, j = m, n
    while i > 0 and j > 0:
        if seq1[i-1] == seq2[j-1]:
            lcs.append((i-1, j-1, seq1[i-1]))
            i -= 1
            j -= 1
        elif dp[i-1][j] > dp[i][j-1]:
            i -= 1
        else:
            j -= 1

    lcs.reverse()
    
    return len(lcs)

In [10]:
print(longest_sub("matematyka", "informatyka"))

6


In [11]:
print(longest_sub("banan", "ananas"))

4


In [12]:
print(longest_sub("los", "kloc"))

2


- Levenshtein

In [13]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros((size_x, size_y))
    for x in range(size_x):
        matrix[x, 0] = x
        
    for y in range(size_y):
        matrix[0, y] = y
    
    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(matrix[x-1, y] + 1, matrix[x-1, y-1], matrix[x, y-1] + 1)
                
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1)
                
    result = int(matrix[size_x - 1, size_y - 1])
    print(matrix)
    
    return result

In [14]:
print(levenshtein("wersacja", "serwacja"))

[[0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [1. 1. 2. 3. 3. 4. 5. 6. 7.]
 [2. 2. 1. 2. 3. 4. 5. 6. 7.]
 [3. 3. 2. 1. 2. 3. 4. 5. 6.]
 [4. 3. 3. 2. 2. 3. 4. 5. 6.]
 [5. 4. 4. 3. 3. 2. 3. 4. 5.]
 [6. 5. 5. 4. 4. 3. 2. 3. 4.]
 [7. 6. 6. 5. 5. 4. 3. 2. 3.]
 [8. 7. 7. 6. 6. 5. 4. 3. 2.]]
2


In [15]:
print(levenshtein("matematyka", "informatyka"))

[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
 [ 1.  1.  2.  3.  4.  5.  5.  6.  7.  8.  9. 10.]
 [ 2.  2.  2.  3.  4.  5.  6.  5.  6.  7.  8.  9.]
 [ 3.  3.  3.  3.  4.  5.  6.  6.  5.  6.  7.  8.]
 [ 4.  4.  4.  4.  4.  5.  6.  7.  6.  6.  7.  8.]
 [ 5.  5.  5.  5.  5.  5.  5.  6.  7.  7.  7.  8.]
 [ 6.  6.  6.  6.  6.  6.  6.  5.  6.  7.  8.  7.]
 [ 7.  7.  7.  7.  7.  7.  7.  6.  5.  6.  7.  8.]
 [ 8.  8.  8.  8.  8.  8.  8.  7.  6.  5.  6.  7.]
 [ 9.  9.  9.  9.  9.  9.  9.  8.  7.  6.  5.  6.]
 [10. 10. 10. 10. 10. 10. 10.  9.  8.  7.  6.  5.]]
5


In [16]:
print(levenshtein("banan", "ananas"))

[[0. 1. 2. 3. 4. 5. 6.]
 [1. 1. 2. 3. 4. 5. 6.]
 [2. 1. 2. 2. 3. 4. 5.]
 [3. 2. 1. 2. 2. 3. 4.]
 [4. 3. 2. 1. 2. 2. 3.]
 [5. 4. 3. 2. 1. 2. 3.]]
3


In [17]:
print(levenshtein("los", "kloc"))

[[0. 1. 2. 3. 4.]
 [1. 1. 1. 2. 3.]
 [2. 2. 2. 1. 2.]
 [3. 3. 3. 2. 2.]]
2


5. Korzystając z gotowego tokenizera dokonaj podziału załączonego tekstu na tokeny.

In [18]:
with open("romeo-i-julia-700.txt", encoding="utf-8") as f:
        text = f.read()

In [19]:
text_list = text.split('\n')
token_list = []
n_tokens = 0
n_lines = 0

for text in text_list:
    new_tokens = nltk.word_tokenize(text)
    token_list.append(new_tokens)
    n_tokens += len(new_tokens)
    n_lines += 1

In [20]:
print(f"Ilość tokenów w analizowanym tekście: {n_tokens}")

Ilość tokenów w analizowanym tekście: 2290


6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.

In [21]:
def del_3percent(text_tokens):
    tkns = deepcopy(text_tokens)
    tkns_all = []
    cnt_del = 0
    while cnt_del < int(n_tokens*0.03):
        i = random.randint(0, n_lines-1)
        line = tkns[i]
        if len(line) > 0:
            j = random.randint(0, len(line)-1)
        else:
            continue
        tkns[i].remove(tkns[i][j])
        cnt_del += 1
        
    for line in tkns:
        if len(line) > 0:
            tkns_all.extend(line)
    
    return tkns, tkns_all

In [22]:
token_list1, tokens_all1 = del_3percent(token_list)
print(f"Ilość tokenów po usunięciu 3%: {len(tokens_all1)}")

Ilość tokenów po usunięciu 3%: 2222


In [23]:
token_list2, tokens_all2 = del_3percent(token_list)
print(f"Ilość tokenów po usunięciu 3%: {len(tokens_all2)}")

Ilość tokenów po usunięciu 3%: 2222


In [24]:
def save_to_txt(filename, text_tokens):
    pnct_lst = [".", ",", ";", ";", "?", "!"]
    with open(filename, "w", encoding="utf8") as f:
        for sublist in text_tokens:
            if not sublist:
                f.write("\n")
            else:
                for i, token in enumerate(sublist):
                    if i == 0:
                        f.write(token)
                    elif token in pnct_lst:
                        f.write(token + " ")
                    else:
                        prev_token = sublist[i-1]
                        if prev_token in pnct_lst:
                            f.write(token)
                        else:
                            f.write(" " + token)
                f.write("\n")

In [25]:
save_to_txt("file1.txt", token_list1)
save_to_txt("file2.txt", token_list2)

7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów. Traktujemy teraz token (wyraz) jako podstawową, niepodzielną jednostkę ciągu.

In [26]:
print(longest_sub(tokens_all1, tokens_all2))

2157


8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff, tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku. Traktujemy teraz całą linię jako podstawową, niepodzielną jednostkę ciągu.

In [27]:
def longest_sub(seq1, seq2):
    m, n = len(seq1), len(seq2)
    dp = [[0] * (n+1) for _ in range(m+1)]

    for i in range(1, m+1):
        for j in range(1, n+1):
            if seq1[i-1] == seq2[j-1]:
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])

    lcs = []
    i, j = m, n
    while i > 0 and j > 0:
        if seq1[i-1] == seq2[j-1]:
            lcs.append((i-1, j-1, seq1[i-1]))
            i -= 1
            j -= 1
        elif dp[i-1][j] > dp[i][j-1]:
            i -= 1
        else:
            j -= 1

    lcs.reverse()
    
    return lcs

In [28]:
def diff(filename1, filename2):
    with open(filename1, encoding="utf-8") as f1:
        text1 = list(f1)
    with open(filename2, encoding="utf-8") as f2:
        text2 = list(f2)
    lcs = longest_sub(text1, text2)
    
    n_text1 = len(text1)
    n_text2 = len(text2)
    lcs_indices = [i1 for i1, _, _ in lcs]
    diff_indices = [i for i in range(n_text1) if i not in lcs_indices]
    
    for i in diff_indices:
        print(f"< {i} | {text1[i]}")
        print(f"> {i} | {text2[i]}")
    
    return lcs

9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

In [29]:
lcs = diff("file1.txt", "file2.txt")

< 3 | tłum. Józef Paszkowski

> 3 | .Józef Paszkowski

< 18 | * LAURENTY — ojciec franciszkanin

> 18 | LAURENTY — ojciec franciszkanin

< 20 | * — służący Romea

> 20 | * BALTAZAR — służący Romea

< 28 | * MONTEKI — małżonka Montekiego

> 28 | * PANI MONTEKI — małżonka Montekiego

< 31 | MARTA — mamka Julii

> 31 | * MARTA — mamka Julii

< 32 | * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i inne osoby. 

> 32 | * Obywatele weroneńscy różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i inne osoby. 

< 33 | 

> 33 | 

< 37 | Rzecz odbywa się przez większą część sztuki w Weronie, przez część aktu w Mantui. 

> 37 | Rzecz odbywa się przez większą część sztuki w Weronie, przez część piątego aktu w Mantui. 

< 47 | Do nowej zbrodni pchają złości dawne, 

> 47 | Do nowej zbrodni pchają dawne, 

< 57 | Aż ją dzieci zgon przedwczesny, 

> 57 | Aż ją zakończy dzieci zgon przedwczesny, 

< 77 | Dalip